In [1]:
import pandas as pd
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r"C:\Users\hp\Desktop\AIML\insurance1.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
from sklearn.model_selection import train_test_split
X = df.drop('bought_insurance',axis=1)
y = df['bought_insurance']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [4]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age']/100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age']/100

In [5]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

model.fit(df[['age', 'affordibility']], df.bought_insurance, epochs=2000)



Epoch 1/2000


1/1 [==============================] - 3s 3s/step - loss: 14.8571 - accuracy: 0.5000
Epoch 2/2000
1/1 [==============================] - 0s 24ms/step - loss: 14.8418 - accuracy: 0.5000
Epoch 3/2000
1/1 [==============================] - 0s 15ms/step - loss: 14.8264 - accuracy: 0.5000
Epoch 4/2000
1/1 [==============================] - 0s 17ms/step - loss: 14.8111 - accuracy: 0.5000
Epoch 5/2000
1/1 [==============================] - 0s 24ms/step - loss: 14.7957 - accuracy: 0.5000
Epoch 6/2000
1/1 [==============================] - 0s 23ms/step - loss: 14.7804 - accuracy: 0.5000
Epoch 7/2000
1/1 [==============================] - 0s 32ms/step - loss: 14.7650 - accuracy: 0.5000
Epoch 8/2000
1/1 [==============================] - 0s 9ms/step - loss: 14.7496 - accuracy: 0.5000
Epoch 9/2000
1/1 [==============================] - 0s 16ms/step - loss: 14.7343 - accuracy: 0.5000
Epoch 10/2000
1/1 [==============================] - 0s 17ms/step - loss: 14.7189 - accuracy: 0.500

In [6]:
model.evaluate(X_test_scaled, y_test)

1/1 [==============================] - 0s 350ms/step - loss: 1.2158 - accuracy: 0.1667


[1.215779423713684, 0.1666666716337204]

In [7]:
y_predicted = model.predict(X_test_scaled)
y_predicted

1/1 [==============================] - 0s 150ms/step


array([[0.23018782],
       [0.25515288],
       [0.25677478],
       [0.25734052],
       [0.25783616],
       [0.2324923 ]], dtype=float32)

In [8]:
coef,intercept = model.get_weights()
coef,intercept

(array([[0.03702733],
        [0.13667567]], dtype=float32),
 array([-1.2165079], dtype=float32))

In [9]:
def sigmoid(X):
    import math
    return 1/(1+math.exp(-X))

In [10]:
sigmoid(10)

0.9999546021312976

In [11]:
def prediction_function(age,affordability):
   weighted_sum = coef[0]*age + coef[1]*affordability + intercept
   return sigmoid(weighted_sum)

In [12]:
X_test_scaled

,age,affordibility
16,0.25,0
26,0.23,1
4,0.46,1
25,0.54,1
9,0.61,1
7,0.60,0


In [13]:
y_predicted


array([[0.23018782],
       [0.25515288],
       [0.25677478],
       [0.25734052],
       [0.25783616],
       [0.2324923 ]], dtype=float32)

In [14]:
prediction_function(0.55,0)

C:\Users\hp\AppData\Local\Temp\ipykernel_26112\59466422.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1/(1+math.exp(-X))


0.23216208158002175

In [15]:
def sigmoid_numpy(X):
    return 1/(1+np.exp(-X))

In [16]:
def log_loss(y_true,y_predicted):
    epsilon = 1e-18
    y_predicted_new =[max(i,epsilon) for i in y_predicted]
    y_predicted_new =[min(i,1-epsilon) for i in y_predicted_new]
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(y_predicted_new))


In [30]:
def Gradient_Descent(age,affordibility,y_true,epochs):
    w1=w2=1
    bias=0
    learning_rate = 0.001
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1*age + w2*affordibility + bias
        y_predicted = sigmoid_numpy(weighted_sum)

        loss = log_loss(y_true,y_predicted)
        w1 = w1 - 1/n*(learning_rate*(np.dot(np.transpose(age),(y_predicted-y_true))))
        w2 = w2 - 1/n*(learning_rate*(np.dot(np.transpose(affordibility),(y_predicted-y_true))))
        bias = bias - learning_rate*(np.mean(y_predicted-y_true))

        print(f'Epoch:{i} w1:{w1} w2:{w2} bias:{bias} loss:{loss}')

In [31]:
Gradient_Descent(df['age'],df['affordibility'],df['bought_insurance'],5000)

Epoch:0 w1:0.9853928571542153 w2:0.9997500000004151 bias:-0.000499999999383465 loss:6.165349993414768e-10
Epoch:1 w1:0.9707857143119554 w2:0.9995000000009566 bias:-0.0009999999985759638 loss:8.07501354379672e-10
Epoch:2 w1:0.9561785714743166 w2:0.9992500000016629 bias:-0.0014999999975182183 loss:1.0577456697949656e-09
Epoch:3 w1:0.9415714286427377 w2:0.999000000002584 bias:-0.0019999999961325034 loss:1.3857150075333093e-09
Epoch:4 w1:0.9269642858191063 w2:0.9987500000037857 bias:-0.0024999999943168894 loss:1.8156142170780991e-09
Epoch:5 w1:0.9123571430059003 w2:0.9985000000053534 bias:-0.0029999999919376797 loss:2.379209810269716e-09
Epoch:6 w1:0.8977500002063727 w2:0.998250000007399 bias:-0.0034999999888194787 loss:3.1182009620578603e-09
Epoch:7 w1:0.8831428574247951 w2:0.9980000000100685 bias:-0.003999999984732141 loss:4.087338008935876e-09
Epoch:8 w1:0.868535714666778 w2:0.9977500000135525 bias:-0.004499999979373617 loss:5.3585242408393006e-09
Epoch:9 w1:0.8539285719396925 w2:0.9975